In [22]:
from matplotlib.pylab import (figure, semilogx, loglog, xlabel, ylabel, legend, 
                           title, subplot, show, grid)
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import loadmat
import pandas as pd
import sklearn.linear_model as lm
from sklearn import model_selection
import torch
import sys
sys.path.append('../Tools')
from toolbox_02450 import rlr_validate, correlated_ttest, train_neural_net, draw_neural_net

from data_preprocessing import *
dfjoint, dfRec, dfClas = dataPreprocess()

colNamesMeans, colNamesStd, colNamesExt, colNamesOther = getSpecificColNames()
mean = dfjoint.loc[:,colNamesMeans]
sde = dfjoint.loc[:,colNamesStd]
worst = dfjoint.loc[:,colNamesExt]
colNames = colNamesMeans + colNamesStd + colNamesExt

time_discretized = []
for i in range(len(dfRec.iloc[:,0])):
    if dfRec['time'].iloc[i] <= 12:
        time_discretized.append('<1 years')
    elif dfRec['time'].iloc[i] <= 36:
        time_discretized.append('1-3 years')
    elif dfRec['time'].iloc[i] <= 72:
        time_discretized.append('>3-6 years')
    else:
        time_discretized.append('6+ years')

dfRec['time_discretized'] = time_discretized

In [25]:
# Data preprocessing
attributeNames = [u'Offset'] + colNames
X = dfRec.loc[:, colNames]

classLabels = dfRec.loc[:,"time_discretized"]
classNames = sorted(set(dfRec.loc[:,"time_discretized"]))
classDict = dict(zip(classNames, range(4)))
                 
y = np.asarray([classDict[value] for value in classLabels])

N = X.shape[0]
M = len(attributeNames)
# M = M + 1
C = len(classNames)

X = np.concatenate((np.ones((X.shape[0],1)),X),1)


In [28]:
## 2 level cross validation, linear regression model

# Crossvalidation
# Create crossvalidation partition for evaluation
K = 5
CV = model_selection.KFold(K, shuffle=True)
#CV = model_selection.KFold(K, shuffle=False)

# Values of lambda
lambdas = np.power(10.,range(-5,9))

# Initialize variables
#T = len(lambdas)
Error_train = np.empty((K,1))
Error_test = np.empty((K,1))
Error_train_rlr = np.empty((K,1))
Error_test_rlr = np.empty((K,1))
Error_train_nofeatures = np.empty((K,1))
Error_test_nofeatures = np.empty((K,1))
w_rlr = np.empty((M,K))
mu = np.empty((K, M-1))
sigma = np.empty((K, M-1))
w_noreg = np.empty((M,K))
errors_lin_reg = np.empty((K,1))

k=0

In [29]:
print(' ')
print('------------------------------------')
print('Linear regression model:')
print(' ')
k=0
for train_index, test_index in CV.split(X,y):
    
    # extract training and test set for current CV fold
    X_train = X[train_index]
    y_train = y[train_index]
    X_test = X[test_index]
    y_test = y[test_index]
    internal_cross_validation = 10    
    
    opt_val_err, opt_lambda, mean_w_vs_lambda, train_err_vs_lambda, test_err_vs_lambda = rlr_validate(X_train, y_train, lambdas, internal_cross_validation)

    # Standardize outer fold based on training set, and save the mean and standard
    # deviations since they're part of the model (they would be needed for
    # making new predictions) - for brevity we won't always store these in the scripts
    mu[k, :] = np.mean(X_train[:, 1:], 0)
    sigma[k, :] = np.std(X_train[:, 1:], 0)
    
    X_train[:, 1:] = (X_train[:, 1:] - mu[k, :] ) / sigma[k, :] 
    X_test[:, 1:] = (X_test[:, 1:] - mu[k, :] ) / sigma[k, :]
    
    # Save generalization error for later statistics
    errors_lin_reg[k] = opt_val_err
    
    k+=1
    
    print('Outer fold {0}'.format(k))
    print('has optimal lambda: {0}'.format(opt_lambda))
    print('and optimal generalization error: {0}'.format(opt_val_err))
    

 
------------------------------------
Linear regression model:
 
Outer fold 1
has optimal lambda: 1000.0
and optimal generalization error: 1.3854540799222033
Outer fold 2
has optimal lambda: 100000000.0
and optimal generalization error: 1.4067527635681032
Outer fold 3
has optimal lambda: 100000000.0
and optimal generalization error: 1.3245162289677554
Outer fold 4
has optimal lambda: 100000.0
and optimal generalization error: 1.2709179897285563
Outer fold 5
has optimal lambda: 1000.0
and optimal generalization error: 1.2549382097651347
